In [ ]:
# %pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [13]:
# 벡터검색 BM25 키워드 검색을 RRF 알고리즘으로 결합
from typing import List
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever
import os
import warnings
import pickle    # chunk, vectorDB 저장한것 사용
from dotenv import load_dotenv

# 경고메세지 삭제
warnings.filterwarnings('ignore')
load_dotenv()

class FusionRetrieval:
    '''Fusion Retrieval 기법을 구현한 클래스'''
    def __init__(self, documents:List[Document], retriever_k: int=5):
        '''
        Args:
            documents : 전체문서 리스트
            retriever_k : 각 검색 방식마다 반환할 문서 개수
        '''
        self.documents=documents
        self.retriever_k = retriever_k

        # BM25 Retriever 초기화
        self.bm25_retriever = BM25Retriever.from_documents(documents)
        self.bm25_retriever.k = retriever_k
        print('Fusion Retriever 초기화 완료')

    #
    def fusion_retriever(self, question:str, vector_retriever) -> List[Document]:
        '''
        벡터 검색과 BM25 검색 결과를 RRF 알고리즘으로 결합
        Args:
            question : 사용자 질문
            vector_retriever : 벡터검색 retriever 객체
        Returns:
            문서리스트
        '''
        # 벡터검색
        vector_docs = vector_retriever.invoke(question)
        
        # BM25 검색
        bm25_docs = self.bm25_retriever.invoke(question)
        print(f'fusion_retriever 사용')
        print(f'벡터 검색 : {len(vector_docs)}개 문서')
        print(f'BM25 검색 : {len(bm25_docs)}개 문서 ')

        # RRF (Reciprocal Rank Fusion) 점수 계산
        fusion_scores = {}
        
        # 벡터 검색 결과 점수 ????????????????????????????????????????????????????/
        for rank, doc in enumerate(vector_docs) :
            doc_key = doc.page_content[:]
            score = 1/(60+rank)  
            fusion_scores[doc_key] = fusion_scores.get(doc_key, 0) + score

        # BM25 검색 결과 점수 ????????????????????????????????????????????????????/
        for rank, doc in enumerate(bm25_docs) :
            doc_key = doc.page_content[:]
            score = 1/(60+rank)  
            fusion_scores[doc_key] = fusion_scores.get(doc_key, 0) + score


        # 점수로 정렬
        sorted_docs = sorted(
            fusion_scores.items(), key=lambda x: x[1], reverse=True
        )

        # 문서 객체 반환
        result = []
        for doc_text , score in sorted_docs[:self.retriever_k] :
            for doc in self.documents:
                if doc.page_content.startswith(doc_text):
                    result.append(doc)
                    break

        print(f"RRF 통합 결과 : {len(result)}개 문서")
        print(f"RRF는 각 검색 방식의 순위 역수를 합산해서 최종 순서를 결정")
        return result
    
    def get_detail_fusion_info (self, question:str, vector_retriever) -> dict:
        '''
        상세한 fusion retriever 정보 반환
        Args:
            question : 사용자 질문
            vector_retriever: 벡터 검색 vector_retriever 객체
        Returns:
            Fusion retriever 상세정보 dict 
        '''
        vector_docs = vector_retriever.invoke(question)
        bm25_docs =self.bm25_retriever.invoke(question)
        return{
            'vector_search_count' : len(vector_docs),
            'bm25_search_count' : len(bm25_docs),
            'vector_docs' : vector_docs,
            'bm25_docs': bm25_docs
        }

if __name__ == '__main__':
    from langchain_community.document_loaders import TextLoader
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    print('1. 문서준비\n\n')
    loader = TextLoader('document_test2.txt', encoding='utf-8')
    documents = loader.load()

    print('2. chunk 생성')
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = 300, 
        chunk_overlap = 70
    )
    chunks = splitter.split_documents(documents)
    print(f'{len(chunks)}개 chunk 생성\n\n')


    print('3. vector store 준비')
    from langchain_openai import OpenAIEmbeddings
    from langchain_chroma import Chroma

    embeddings = OpenAIEmbeddings(
        model='text-embedding-3-small',
        api_key=os.environ.get('OPENAI_API_KEY')
    )
    vectorstore = Chroma.from_documents(
            chunks,
            embeddings,
            persist_directory='./chroma_db_advance_fusion'
    )
    print(f'vector store 준비 완료\n\n')

    print(f'retriever 생성')
    retriever = vectorstore.as_retriever(search_kwargs = {'k' :3})

    question = 'LangChain의 요소는 무엇인가요?'
    print(f'FusionRetrieval 사용')
    fusion = FusionRetrieval(chunks, 3)
    fusion_docs = fusion.fusion_retriever(question, retriever)
    print(f'fusion 검색결과 : {len(fusion_docs)}개')
    print('검색 결과.....')    
    for fusion_doc in fusion_docs:
        print(fusion_doc.page_content)   

1. 문서준비


2. chunk 생성
2개 chunk 생성


3. vector store 준비
vector store 준비 완료


retriever 생성
FusionRetrieval 사용
Fusion Retriever 초기화 완료
fusion_retriever 사용
벡터 검색 : 3개 문서
BM25 검색 : 2개 문서 
RRF 통합 결과 : 2개 문서
RRF는 각 검색 방식의 순위 역수를 합산해서 최종 순서를 결정
fusion 검색결과 : 2개
검색 결과.....
LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다.
        
        LangChain의 주요 구성 요소:
        1. Models: 다양한 LLM 제공자(OpenAI, Anthropic, Google 등)와 통합
        2. Prompts: 프롬프트 템플릿 관리 및 최적화
        3. Chains: 여러 구성 요소를 연결하는 파이프라인
        4. Memory: 대화 맥락을 유지하기 위한 메모리 시스템
4. Memory: 대화 맥락을 유지하기 위한 메모리 시스템
        5. Indexes: 문서 검색을 위한 인덱싱 도구
        6. Agents: 도구를 사용하여 복잡한 작업을 수행하는 에이전트
        
        LangChain Expression Language (LCEL)은 체인을 구성하는 선언적 방식으로,
        파이프(|) 연산자를 사용하여 컴포넌트들을 직관적으로 연결할 수 있습니다.
